In [1]:
import numpy as np
import pandas as pd
from modlamp.descriptors import PeptideDescriptor, GlobalDescriptor
import regex as re
Root = "C:/Users/JJvRijn/Documents/Python_Scripts"

In [5]:
DBAASP_data = pd.read_csv(Root + "/" + "Programing/Data/DBAASP_complete.csv")

C:\Users\JJvRijn\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (16,17,23,24,31,32,38,53,54,57,63,64,65,66,67,68,71,76,79) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [23]:
#extract the usefull columns
DBAASP_usefull = DBAASP_data[["ID", "SEQUENCE", "HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS VALUE",
                              "HEMOLITIC CYTOTOXIC ACTIVITY - CONCENTRATION",
                             "HEMOLITIC CYTOTOXIC ACTIVITY - UNIT", 'HEMOLITIC CYTOTOXIC ACTIVITY - TARGET CELL' 
                              ]]


In [26]:
DBAASP_data['HEMOLITIC CYTOTOXIC ACTIVITY - TARGET CELL'].value_counts()

Human erythrocytes                             10272
Mouse erythrocytes                               709
Sheep erythrocytes                               662
Horse erythrocytes                               584
Rabbit erythrocytes                              537
                                               ...  
Human stromal prostate fibroblasts G1947           1
Human prostatic cells PNT1A                        1
Bone marrow–derived dendritic cells (BMDCs)        1
Avian hepatoma cells LMH-D2                        1
Mussel plasma                                      1
Name: HEMOLITIC CYTOTOXIC ACTIVITY - TARGET CELL, Length: 199, dtype: int64

In [38]:
#make the data workable
#by removing unknown values and empty values
DBAASP_usefull = DBAASP_usefull.dropna(thresh=3)
DBAASP_usefull.reset_index(drop=True)
human = []
for j in DBAASP_usefull.index:
    if "human" in DBAASP_usefull[
    'HEMOLITIC CYTOTOXIC ACTIVITY - TARGET CELL'][j].lower():
        human.append(j)
DBAASP_usefull = DBAASP_usefull.loc[human]
DBAASP_usefull.reset_index(drop=True)
#MEC isnt universally defined so is removed as correct input is near impossible
#without processing the indvidual papers
DBAASP_usefull = DBAASP_usefull[DBAASP_usefull[
    "HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS VALUE"] != 'MEC']
DBAASP_usefull = DBAASP_usefull[DBAASP_usefull[
    "HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS VALUE"] != '-']
DBAASP_usefull = DBAASP_usefull[DBAASP_usefull[
    "HEMOLITIC CYTOTOXIC ACTIVITY - CONCENTRATION"] != 'nan']
DBAASP_usefull["HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS VALUE"
              ] = DBAASP_usefull["HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS VALUE"].replace([
    "EC50", "IC50", 'LD50', 'CC50', 'LC50', 'MHC', 'LC90', 'IC90', 'ED50', 'LD90']
    , ["50%",'50%', '50%', '50%', '50%', '15%', '90%', '90%', '50%', '90%'])

DBAASP_usefull = DBAASP_usefull.dropna(subset=["HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS VALUE"])
DBAASP_usefull = DBAASP_usefull.dropna(subset=["HEMOLITIC CYTOTOXIC ACTIVITY - CONCENTRATION"])
DBAASP_usefull = DBAASP_usefull.reset_index(drop=True)

In [60]:
DBAASP_usefull[2642:2644]

,ID,SEQUENCE,HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS VALUE,HEMOLITIC CYTOTOXIC ACTIVITY - CONCENTRATION,HEMOLITIC CYTOTOXIC ACTIVITY - UNIT,HEMOLITIC CYTOTOXIC ACTIVITY - TARGET CELL,MW,CONCENTRATION_µM,conc_num,Lysis_value
2642,5499,AGYLLGKINLKALAALAKKIL,80% Hemolysis,512,µg/ml,Human erythrocytes,2181.76,234.672925,512,80
2643,5499,AGYLLGKINLKALAALAKKIL,LC,128,µg/ml,Human umbilical vein endothelial cells HUVEC,2181.76,58.668231,128,NaN


In [55]:
#remove unnatural Amino acids
NAA = ["A", "R", "N", "D", "C", "Q", "E", "G", "H", "I", "L", "K", "M", "F", "P", "S", "U",
       "T", "W", "Y", "V", " "]
unvalid =[]
for j in DBAASP_usefull.index:
    if (not all ([NAA_seq in NAA for NAA_seq in DBAASP_usefull.SEQUENCE[j]])):
        unvalid.append(j)
DBAASP_usefull = DBAASP_usefull.drop(unvalid)
DBAASP_usefull = DBAASP_usefull.reset_index(drop=True)
#add molweight to covert ug/ml to uM
glob = GlobalDescriptor(DBAASP_usefull.SEQUENCE.tolist())
glob.calculate_MW(amide=True)
DBAASP_usefull['MW'] = glob.descriptor
#see what concentrations are in µg/ml instead of µM
mask = DBAASP_usefull["HEMOLITIC CYTOTOXIC ACTIVITY - UNIT"] == 'µg/ml'
#extract numerical to remove >, <, <= etc.
DBAASP_usefull["CONCENTRATION_µM"] = 0
for o in range(len(DBAASP_usefull)):
    if "-" in str(DBAASP_usefull["HEMOLITIC CYTOTOXIC ACTIVITY - CONCENTRATION"][o]):
         DBAASP_usefull.loc[o, "conc_num"] = re.findall(r"[-+]?\d*\.\d+|\d+",str(DBAASP_usefull[
        "HEMOLITIC CYTOTOXIC ACTIVITY - CONCENTRATION"][o]))[1]
    else:
        DBAASP_usefull.loc[o, "conc_num"] = re.findall(r"[-+]?\d*\.\d+|\d+",str(DBAASP_usefull[
            "HEMOLITIC CYTOTOXIC ACTIVITY - CONCENTRATION"][o]))[0]
#calculate the µM
DBAASP_usefull["CONCENTRATION_µM"] = 0
DBAASP_usefull.loc[mask, "CONCENTRATION_µM"] = ((DBAASP_usefull[
    "conc_num"][mask]).astype(float)*1000) / DBAASP_usefull[mask]['MW']
#fill in values already in µM
mask2 = ~mask
DBAASP_usefull.loc[mask2, "CONCENTRATION_µM"] = DBAASP_usefull["conc_num"][mask2].astype(float)
DBAASP_usefull = DBAASP_usefull.reset_index(drop=True)

In [70]:
#Extracting int for Lysis_value
mask3 = []
for i in DBAASP_usefull.index:
    try:
        if "-" in str(DBAASP_usefull["HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS VALUE"][o]):
            DBAASP_usefull.loc[i, "Lysis_value"] = re.findall(r"[-+]?\d*\.\d+|\d+",str(DBAASP_usefull[
                "HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS VALUE"][i]))[1]       
        else:
            DBAASP_usefull.loc[i, "Lysis_value"] = re.findall(r"[-+]?\d*\.\d+|\d+",str(DBAASP_usefull[
                "HEMOLITIC CYTOTOXIC ACTIVITY - LYSIS VALUE"][i]))[0]
    except:
        mask3.append(i)
DBAASP_usefull = DBAASP_usefull.drop(unvalid)
DBAASP_usefull = DBAASP_usefull.reset_index(drop=True)

In [71]:
#happenn formating
DBAASP_usefull["toxic"] = "~"
for v in range(len(DBAASP_usefull)):
    if float(DBAASP_usefull["Lysis_value"][v]) <= 5:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) >= 30:
            DBAASP_usefull.loc[v, "toxic"] = 0        
    if 5 <= float(DBAASP_usefull["Lysis_value"][v]) <= 10:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) >= 60:
            DBAASP_usefull.loc[v, "toxic"] = 0   
    if 10 <= float(DBAASP_usefull["Lysis_value"][v]) <= 15:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) >= 90:
            DBAASP_usefull.loc[v, "toxic"] = 0
    if 15 <= float(DBAASP_usefull["Lysis_value"][v]) <= 20:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) >= 120:
            DBAASP_usefull.loc[v, "toxic"] = 0
    if 20 <= float(DBAASP_usefull["Lysis_value"][v]) <= 25:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) >= 180:
            DBAASP_usefull.loc[v, "toxic"] = 0  
    if 25 <= float(DBAASP_usefull["Lysis_value"][v]) <= 30:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) >= 180:
            DBAASP_usefull.loc[v, "toxic"] = 0  
    if 30 <= float(DBAASP_usefull["Lysis_value"][v]) <= 35:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) >= 210:
            DBAASP_usefull.loc[v, "toxic"] = 0         
    if 35 <= float(DBAASP_usefull["Lysis_value"][v]) <= 40:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) >= 240:
            DBAASP_usefull.loc[v, "toxic"] = 0  
    if 40 <= float(DBAASP_usefull["Lysis_value"][v]) <= 45:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) >= 270:
            DBAASP_usefull.loc[v, "toxic"] = 0  
    if float(DBAASP_usefull["Lysis_value"][v]) >= 50:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) <= 300:
            DBAASP_usefull.loc[v, "toxic"] = 1
    if float(DBAASP_usefull["Lysis_value"][v]) >= 55:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) <= 330:
            DBAASP_usefull.loc[v, "toxic"] = 1
    if float(DBAASP_usefull["Lysis_value"][v]) >= 60:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) <= 360:
            DBAASP_usefull.loc[v, "toxic"] = 1
    if float(DBAASP_usefull["Lysis_value"][v]) >= 65:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) <= 390:
            DBAASP_usefull.loc[v, "toxic"] = 1 
    if float(DBAASP_usefull["Lysis_value"][v]) >= 70:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) <= 420:
            DBAASP_usefull.loc[v, "toxic"] = 1 
    if float(DBAASP_usefull["Lysis_value"][v]) >= 75:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) <= 450:
            DBAASP_usefull.loc[v, "toxic"] = 1 
    if float(DBAASP_usefull["Lysis_value"][v]) >= 80:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) <= 480:
            DBAASP_usefull.loc[v, "toxic"] = 1 
    if float(DBAASP_usefull["Lysis_value"][v]) >= 85:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) <= 510:
            DBAASP_usefull.loc[v, "toxic"] = 1 
    if float(DBAASP_usefull["Lysis_value"][v]) >= 90:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) <= 540:
            DBAASP_usefull.loc[v, "toxic"] = 1
    if float(DBAASP_usefull["Lysis_value"][v]) >= 95:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) <= 570:
            DBAASP_usefull.loc[v, "toxic"] = 1
    if float(DBAASP_usefull["Lysis_value"][v]) >= 100:
        if (DBAASP_usefull["CONCENTRATION_µM"][v]) <= 600:
            DBAASP_usefull.loc[v, "toxic"] = 1 

In [76]:
#remove rows were toxicity is unclear
DBAASP_usefull = DBAASP_usefull[DBAASP_usefull["toxic"] != '~']
DBAASP_usefull = DBAASP_usefull.reset_index(drop=True)
#merge same peptides
#where majority vote decides toxicity and a draw gives "~"
peptides = DBAASP_usefull["SEQUENCE"].unique()
DBAASP_final = pd.DataFrame(peptides, columns=['seq'])
for a in range(len(DBAASP_usefull)):
    for n in range(len(peptides)):
        toxic = []
        toxic_count = 0
        if DBAASP_usefull["SEQUENCE"][a] == peptides[n]:
            toxic.append(DBAASP_usefull["toxic"][a])
            toxic_count = sum(toxic)/len(toxic)
            if toxic_count == len(toxic)/2:
                DBAASP_final.loc[n, "toxic"] = "~"
            if toxic_count < len(toxic)/2:
                DBAASP_final.loc[n, "toxic"] = 0
            if toxic_count > len(toxic)/2:
                DBAASP_final.loc[n, "toxic"] = 1

In [77]:
DBAASP_final["toxic"].value_counts()

1.0    2192
0.0    2070
Name: toxic, dtype: int64

In [156]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef
from lazypredict.Supervised import LazyClassifier
from DNN_features import *
test2 = AAC_df(DBAASP_final.seq)
train_X, val_X, train_y, val_y = train_test_split(test2, DBAASP_final.toxic, random_state = 445)
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=matthews_corrcoef)
#models_train,predictions_train = clf.fit(train_X, train_X, train_y, train_y)
models,predictions = clf.fit(train_X, val_X, train_y, val_y)
models

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:04<00:00,  6.22it/s]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,matthews_corrcoef,Time Taken
Model,,,,,,
ExtraTreesClassifier,0.83,0.83,0.83,0.83,0.65,0.38
RandomForestClassifier,0.82,0.82,0.82,0.82,0.64,0.46
LGBMClassifier,0.80,0.80,0.80,0.80,0.60,0.13
XGBClassifier,0.80,0.80,0.80,0.80,0.59,0.19
NuSVC,0.78,0.78,0.78,0.78,0.56,0.29
LabelPropagation,0.78,0.78,0.78,0.78,0.56,0.49
LabelSpreading,0.78,0.78,0.78,0.78,0.55,0.61
BaggingClassifier,0.78,0.77,0.77,0.78,0.55,0.15
SVC,0.77,0.77,0.77,0.77,0.53,0.21


In [79]:
DBAASP_final.to_csv(Root + "/Programing/Data/DBAASP_clean.csv",index = False, header=True)

In [157]:
DBAASP_final.toxic.value_counts()

0.00    1652
1.00    1461
Name: toxic, dtype: int64